In [57]:
from mpl_toolkits.axes_grid1 import ImageGrid
from numpy import array
import numpy as np
#from statistics import mean as mean
#from statistics import stdev as stddev
import matplotlib.pyplot as plt
import time
import matplotlib.cm as cmap

In [58]:
N_digits = 60000
D_Size = 784
D_bytes = N_digits * D_Size
D_rows = N_digits * 28

MNIST_file = open ("train-images.idx3-ubyte","rb")
z = MNIST_file.read (16 + D_bytes)
len (z)
digits_raw = z[16:(16 + D_bytes)]
w = np.empty (shape=(D_rows, 28))
print (len (digits_raw))
# convert the bytes to Numpy floats for Keras
for i in range (0, D_rows) :
    for j in range (0,28) :
        w[i, j] = float (digits_raw[i*28 + j])/255
Training = w.reshape (N_digits, 28, 28)
np.shape (Training)
counter = 0
spikearray = []
#print(Training[0])
for i in range(0,len(Training)):
    for j in range(0,28):
        for k in range(0,28):
            if Training[i,j,k] >= 0.5:
                Training[i,j,k] = 10*counter
                spikearray.append(Training[i,j,k])
            else:
                Training[i,j,k] = 0
            counter+=1
            
print(Training[1])

47040000
[[    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.]
 [    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
      0.     0.     0.     0.     0.     0.  9120.  9130.  9140.     0.
      0.     0.     0.     0.     0

In [59]:
ALIF_neuron_parameters = {
    'v_rest':   -70.6,  # Resting membrane potential in mV.
    'cm':         0.281,  # Capacity of the membrane in nF
    'tau_m':     9.3667,  # Membrane time constant in ms.
    'tau_refrac': 0.0,  # Duration of refractory period in ms.
    'tau_syn_E':  5.0,  # Decay time of the excitatory synaptic conductance in ms.
    'tau_syn_I':  5.0,  # Decay time of the inhibitory synaptic conductance in ms.
    'e_rev_E':    0.0,  # Reversal potential for excitatory input in mV
    'e_rev_I':  -80.0,  # Reversal potential for inhibitory input in mV
    'v_thresh': -50.4,  # Spike threshold in mV.
    'v_reset':  -70.6,  # Reset potential after a spike in mV.
    'i_offset':   0.0,  # Offset current in nA
}
ALIF_neuron_parameters_exc = {
    'v_rest':   -70.6,  # Resting membrane potential in mV.
    'cm':         0.281,  # Capacity of the membrane in nF
    'tau_m':     9.3667,  # Membrane time constant in ms.
    'tau_refrac': 0.0,  # Duration of refractory period in ms.
    'tau_syn_E':  5.0,  # Decay time of the excitatory synaptic conductance in ms.
    'tau_syn_I':  5.0,  # Decay time of the inhibitory synaptic conductance in ms.
    'e_rev_E':    80.0,  # Reversal potential for excitatory input in mV
    'e_rev_I':  0.0,  # Reversal potential for inhibitory input in mV
    'v_thresh': -50.4,  # Spike threshold in mV.
    'v_reset':  -70.6,  # Reset potential after a spike in mV.
    'i_offset':   0.0,  # Offset current in nA
}

In [60]:
import spynnaker8 as sim
import pyNN.utility.plotting as plot
import matplotlib.pyplot as plt


batchsize = 10
simtime = 28*28*10*batchsize
n_neurons = simtime
sim.setup(timestep=1)

pre_alif_pop = sim.Population(n_neurons, sim.IF_cond_exp(**ALIF_neuron_parameters),initial_values={'v': ALIF_neuron_parameters["v_rest"]},label="PreAlif")
post_alif_pop = sim.Population(n_neurons, sim.IF_cond_exp(**ALIF_neuron_parameters_exc),initial_values={'v': ALIF_neuron_parameters_exc["v_rest"]},label="PostAlif")

pre_alif_pop.record(["spikes",'v','gsyn_inh','gsyn_exc'])
post_alif_pop.record(["spikes",'v','gsyn_exc','gsyn_inh'])
%matplotlib inline
inp = sim.Population(simtime,
                   sim.SpikeSourceArray(spikearray),
                   label="Input"
                  )

2022-05-20 23:19:56 WARNING: Calling setup a second time causes the previous simulator to be stopped and cleared.
2022-05-20 23:19:58 INFO: Caching data for segment -1
2022-05-20 23:21:24 WARNING: The simulation has not yet run, therefore spikes cannot be retrieved, hence the list will be empty
2022-05-20 23:21:24 WARNING: Population: Input Does not support units for spikes
2022-05-20 23:21:24 INFO: Caching data for segment -1
2022-05-20 23:21:26 WARNING: The simulation has not yet run, therefore spikes cannot be retrieved, hence the list will be empty
2022-05-20 23:21:26 INFO: Resetting
2022-05-20 23:21:27 WARNING: /home/bbpnrsoa/reports has 23 old reports that have not been closed
2022-05-20 23:21:29 INFO: Read cfg files: /home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/bbpnrsoa/.spynnaker.cfg
2022-05-20 23:21:29 INFO: Will search these location

['/home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/bbpnrsoa/sPyNNaker/lib/python3.8/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/bbpnrsoa/.spynnaker.cfg']


2022-05-20 23:21:29 WARNING: /home/bbpnrsoa/reports has 23 old reports that have not been closed
2022-05-20 23:21:29 INFO: Setting time scale factor to 1.
2022-05-20 23:21:29 INFO: Setting machine time step to 1000 micro-seconds.


In [61]:
#training = sim.Population(n_neurons, sim.SpikeSourceArray(spike_times=xor),label="Training")
stdp_initial_weights = sim.RandomDistribution(distribution='normal_clipped',low=0,high=1, mu=0.5, sigma=0.3)
sim.Projection(inp,pre_alif_pop,sim.OneToOneConnector(),
              synapse_type=sim.StaticSynapse(weight=5.0,delay=2.0))
sim.Projection(inp,post_alif_pop,sim.OneToOneConnector(),
              synapse_type=sim.StaticSynapse(weight=5.0,delay=10.0))

sim.Projection(post_alif_pop,post_alif_pop,sim.OneToOneConnector(),synapse_type=sim.StaticSynapse(weight=2.0),receptor_type = 'excitatory')#recurrent
sim.Projection(pre_alif_pop,pre_alif_pop,sim.OneToOneConnector(),synapse_type=sim.StaticSynapse(weight=2.0),receptor_type = 'inhibitory')#recurrent

projection from pre PreAlif to post PreAlif with connector OneToOneConnector()

In [62]:
timing_rule = sim.SpikePairRule(tau_plus=20.0,tau_minus=20.0,A_plus=0.5,A_minus=0.5)
weight_rule = sim.AdditiveWeightDependence(w_max=5.0,w_min=0.0)
stdp_model = sim.STDPMechanism(timing_dependence=timing_rule,weight_dependence=weight_rule,
                              weight=stdp_initial_weights,delay=5.0)
stdp_projection = sim.Projection(pre_alif_pop,post_alif_pop,sim.OneToOneConnector(),synapse_type=stdp_model,receptor_type = 'excitatory')

In [ ]:
sim.run(50)
initial_weights = stdp_projection.get(["weight"],"list", with_address=False)

2022-05-20 23:21:40 INFO: Starting execution process
2022-05-20 23:21:40 INFO: Simulating for 50 1.0ms timesteps using a hardware timestep of 1000us
Adding Splitter selectors where appropriate
|0%                          50%                         100%|
2022-05-20 23:21:46 INFO: Time 0:00:00.315364 taken by SpynnakerSplitterSelector
Adding delay extensions as required
|0%                          50%                         100%|
2022-05-20 23:21:46 INFO: Time 0:00:00.108250 taken by DelaySupportAdder


In [ ]:
print(initial_weights)
sim.run(simtime)
pre_neo = pre_alif_pop.get_data(variables=["spikes"])
pre_spikes = pre_neo.segments[0].spiketrains
post_neo = post_alif_pop.get_data(variables=["spikes"])
post_spikes = post_neo.segments[0].spiketrains
pre_v = pre_alif_pop.get_data("v")
post_v = post_alif_pop.get_data("v")
print(stdp_projection.getWeights())



In [ ]:
exc_v = post_alif_pop.get_data("v")
exc_ge = post_alif_pop.get_data('gsyn_exc')
exc_gi = post_alif_pop.get_data('gsyn_inh')
inh_v = pre_alif_pop.get_data("v")
inh_ge = pre_alif_pop.get_data('gsyn_exc')
inh_gi = pre_alif_pop.get_data('gsyn_inh')

plot.Figure(
    #plot.Panel(inp_spikes.segments[0].spiketrains,yticks=True,xticks=True,xlabel="Time"),
    plot.Panel(pre_spikes,yticks=True,xticks=True,xlabel="Time"),
    plot.Panel(post_spikes,yticks=True,xticks=True,xlabel="Time"),
    #plot.Panel(exc_v.segments[0].filter(name='v')[0],yticks=True,xticks=True,legend=None,ylim=(-70,-50)),
    plot.Panel(exc_v.segments[0].filter(name='v')[0],yticks=True,xticks=True,legend=None),
    plot.Panel(inh_v.segments[0].filter(name='v')[0],yticks=True,xticks=True,legend=None),
    plot.Panel(exc_ge.segments[0].filter(name='gsyn_exc')[0],yticks=True,xticks=True,legend=None),
    plot.Panel(inh_gi.segments[0].filter(name='gsyn_inh')[0],yticks=True,xticks=True,legend=None),
    plot.Panel(exc_ge.segments[0].filter(name='gsyn_exc')[0],yticks=True,xticks=True,legend=None),
    plot.Panel(inh_gi.segments[0].filter(name='gsyn_inh')[0],yticks=True,xticks=True,legend=None)
).save("MNIST_Plot")

In [ ]:
weights = stdp_projection.get(["weight"],"list", with_address=False)
weights_address = stdp_projection.get(["weight"],"list")
min_weight = 0
max_weight = 5

In [ ]:
# Visualizing Weights
# 0 1 2 ...
# 10 11 12 ...
# 20 21 22 ...
# ...
# 90 91 92 ...
nRow = int(np.sqrt(n_neurons))
nCol = int(np.sqrt(n_neurons))
fig = plt.figure(figsize=(20, 20))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(nRow, nCol),  # creates nrow x ncol grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 )
wt_imgs = [np.zeros((34,34),dtype=np.float32) for wt_img in range(n_neurons)]
for trgt_nrns in range(n_neurons):
    for tp in weights_address: # format of tuple ['source','target','weight']
        if tp[1]==trgt_nrns:
            wt_imgs[trgt_nrns][tp[0]//34][tp[0]%34] = ((tp[2]-min_weight)/max_weight)*255
    
for ax, im in zip(grid, wt_imgs):
    # Iterating over the grid returns the Axes.
    ax.imshow(im,cmap=cmap.get_cmap('hot_r'))
plt.show()

In [ ]:
import pyNN.spiNNaker as p
import matplotlib.pyplot as plt
import pyNN.utility.plotting as plot
import numpy as np
import seaborn as sns
import os
import sys
import random
%matplotlib inline
class_history = []
n_excitatory_neurons = n_neurons
classes = [0,1,2,3,4,5,6,7,8,9] # i.e. digits 5,6,9
class_selector = []
for i in range(10):
    random.shuffle(classes)
    class_selector+=classes
class_history+=list(class_selector)
# Printing max length of array that can be created
print(sys.maxsize)

# Importing functions for NMNIST Data
import eventvision as ev

# Setting up simulation
p.setup(timestep=1,time_scale_factor=5)
# p.set_number_of_neurons_per_core(neuron_type=p.IF_cond_exp(),max_permitted=150)

num_examples_seen = {}
for c in classes:
    num_examples_seen[c]=0




# input
inp = p.Population(1156,
                   p.SpikeSourceArray(spikearray),
                   label="Input"
                  )
# excitatory
exc = p.Population(n_excitatory_neurons,
                   p.IF_cond_exp(**ALIF_neuron_parameters_exc),
                   initial_values={'v': ALIF_neuron_parameters_exc["v_rest"]},
                   label="Excitatory"
                  )
# inhibitory
inh = p.Population(n_neurons,
                   p.IF_cond_exp(**ALIF_neuron_parameters),
                   initial_values={'v': ALIF_neuron_parameters["v_rest"]},
                   label="Inhibitory"
                  )

# Record Spikes
inp.record(["spikes"])
exc.record(["spikes"])

max_weight = 1
# E -> I
# e_i_weight_rand = p.RandomDistribution(distribution='uniform',low=0.0,high=max_weight)
p.Projection(presynaptic_population = exc,
             postsynaptic_population = inh,
             connector = p.OneToOneConnector(),
             synapse_type = p.StaticSynapse(weight=1,delay=1.0),
             receptor_type = 'excitatory'
            )

# I -> E
i_e_connection_list = []
for i in range(n_neurons):
    for j in range(n_neurons):
        if not i==j:
            i_e_connection_list.append((i,j))
        print(i_e_connection_list[i,j])
print(len(i_e_connection_list))
p.Projection(presynaptic_population = inh,
             postsynaptic_population = exc,
             connector = p.FromListConnector(i_e_connection_list),
             synapse_type = p.StaticSynapse(weight=0.1,delay=1.0),
             receptor_type = 'inhibitory'
            )
# Input -> E
stdp_proj = p.Projection(presynaptic_population = inp,
             postsynaptic_population = exc,
             connector = p.AllToAllConnector(),
             synapse_type = p.StaticSynapse(weight=weights,delay=1.0),
             receptor_type = 'excitatory'
            )

p.run(28*28*10)

inp_spikes = inp.get_data("spikes")
exc_spikes = exc.get_data("spikes")